code mdpc

In [1]:
#code permettant de mettre la matrice génératrice sous forme systématique à partir de la matrice de parité
def dual(f):
    g = copy(f)
    ligneg = len(g.rows())
    colg = len(g.columns())
    d = matrix(GF(2),[[0 for a in range(colg)] for b in range(colg-ligneg)])#sous matrice carré à gauche
    ide = identity_matrix(GF(2),colg-ligneg)
    for i in range(colg-ligneg):
        for j in range(ligneg):
            d[i,j] = -g[j][i+ligneg]#-A transpose à gauche
        for k in range(ligneg,colg):
            d[i,k] = ide[i,k-ligneg]#identité à droite
    return d

In [2]:
#fonction permettant de décaler tous les éléments d'une liste de gauche à droite
def Decalage(l):
    return [l[len(l)-1]] + l[:len(l)-1]

In [3]:
#fonction utilisé pour la génération de la matrice de parité Sk en concaténant 2 sous matrices carrées
#ciculantes 
def MatGen(la,lb):
    taille = len(la)
    a = matrix(GF(2),taille,taille)
    b = matrix(GF(2),taille,taille)
    for i in range(taille):
        for j in range(taille):
            a[i,j] = la[j]
            b[i,j] = lb[j]
        la = Decalage(la) #création de la sous matrice carré à gauche avec un décalage de la première ligne 
        lb = Decalage(lb) #création de la sous matrice carré à droite avec un décalage de la première ligne
    Sk = matrix(GF(2),taille,taille*2) #création de la matrice Sk
    for i in range(taille):
        for j in range(taille):
            Sk[i,j] = a[i,j] #on remplit la matrice Sk à gauche
        for k in range(taille,taille*2):
            Sk[i,k] = b[i,k-taille] #on remplit la matrice Sk à droite
    return a,Sk # on retourne la sous matrice carrée à gauche et la matrice Sk pour simplifier les calculs d'inversion de matrice

In [4]:
#fonction permettant de savoir si une liste contient que des bits 0 ou non.
def Zero(s):
    for el in s:
        if el != 0:
            return False
    return True

In [5]:
#fonction permettant de placer dans une liste les indices pour lesquels la liste passés en paramètres vaut 1.
def test_1(i,liste):
    l = []
    for k in range (len(liste)):
        if liste[k] == 1 :
            l.append(k)
    return l

In [6]:
#fonction permettant de donner le poids d'une liste.
def poids(Syndrome):
    poids = 0
    for i in range (len(Syndrome)):
        if Syndrome[i] == 1 :
            poids = poids + 1
    return poids

In [29]:
def bitflipv1(Sk, MessageChiffre, Seuil):
    y = copy(MessageChiffre[0]) # le message chiffré est defini coomme une matrice de dimension 1 d'où l'indice 0
    Syndrome = y*Sk.transpose()
    while(not Zero(Syndrome)): #condition dès que le syndrome vaut 0 on a donc un message appartenant au code C
        for i in range(len(y)):
            s = 0
            for j in range(len(Syndrome)):
                if Syndrome[j] and Sk.transpose()[i][j]: #Pour tous les indices des colonnes de notre matrice H si nous avons des bits à 1 et de même avce le syndrome alors 
                    s += 1 #on note le nombre de bits 1 en communs
            if s >= Seuil: #si ce nombre de bits en communs dépassent un certain Seuil on modifie le bits du message chiffré
                y[i] += 1%2
        Syndrome = y*Sk.transpose() #on recalcule le syndrome avec notre chiffré modifié
    return y

In [7]:
#version optimisé de l'algorithme de bitflip
def bitflipv2(Sk, MessageChiffre, Seuil, iteration):
    y = copy(MessageChiffre[0]) # le message chiffré est defini coomme une matrice de dimension 1 d'où l'indice 0
    Syndrome = y*Sk.transpose() 
    tour = 0 
    while(not Zero(Syndrome)): #condition dès que le syndrome vaut 0 on a donc un message appartenant au code C
        for i in range(len(y)):
            s = 0
            l = test_1(i,Sk.transpose()[i]) #on note les indices pour lesquels il y'a des 1 dans les colonnes de Sk
            for j in l :
                if Syndrome[j] == 1 : #si pour les mêmes indices nous avons aussi les bits du syndromes qui valent 1
                    s = s+1 #on note le nombre de bits 1 en communs 
            if s >= Seuil : #si ce nombre de bits en communs dépassent un certain Seuil on modifie le bits du message chiffré
                y[i] += 1%2
        tour = tour + 1 #on note à chaque fois le nombre d'itération effectué
        Syndrome = y*Sk.transpose() #on recalcule le syndrome avec notre chiffré modifié
        if tour >= iteration : #au bout d'un certain nombre de tour si nous ne sommes toujours pas sortie du while on estime que nous avons échoué
            return "erreur"
    return y

In [8]:
#version raffinée de l'algorithme du bitflip
def bitflipv3(Sk, MessageChiffre, Seuil, iteration):
    y = copy(MessageChiffre[0]) # le message chiffré est defini coomme une matrice de dimension 1 d'où l'indice 0
    Syndrome = y*Sk.transpose()
    tour = 0
    liste = []
    while(not Zero(Syndrome)): #condition dès que le syndrome vaut 0 on a donc un message appartenant au code C
        for i in range(len(y)):
            s = 0
            l = test_1(i,Sk.transpose()[i]) #on note les indices pour lesquels il y'a des 1 dans les colonnes de Sk
            for j in l :
                if Syndrome[j] == 1 :#si pour les mêmes indices nous avons aussi les bits du syndromes qui valent 1
                    s = s+1 #on note le nombre de bits 1 en communs 
            if s >= Seuil : #si ce nombre de bits en communs dépassent un certain Seuil on modifie le bits du message chiffré
                y[i] += 1%2
        tour = tour + 1 #on note à chaque fois le nombre d'itération effectué
        liste.append(poids(Syndrome)) #après chaque itération on note le poids du syndrome
        if tour >= 3 :
            if liste[tour-1] == liste[tour-2]: #si sur 2 itérations le poids du syndrome ne change pas 
                Seuil = Seuil-1 #on diminue le Seuil
                iteration = iteration+3 #on augmente le nombre d'itération possible 
        Syndrome = y*Sk.transpose() #on recalcule le syndrome avec notre chiffré modifié
        if tour >= iteration : #au bout d'un certain nombre de tour si nous ne sommes toujours pas sortie du while on estime que nous avons échoué
            return "erreur"
    return y

In [9]:
#fonction permettant l'automatisation des tests avec la version optimisée
def qualite_test(n,seuil,tour,erreur,Pk):
    r = 0
    d = 0
    for i in range (n):
        mot = Pk[randint(0,511)]
        for k in range(10):
            mot += Pk[randint(0,511)] # on génère aléatoirement un message mot
        mot2 = copy(mot)
        for j in range(erreur):
            mot[randint(0,1023)] += 1 # on génère aléatoirement le vecteur erreur pour voir notre chiffré
        chiffre = matrix(GF(2),mot)
        message = bitflipv2(H,chiffre,seuil,tour)
        if message == "erreur" :
            d = d+1 
        if message == mot2 :
            r = r+1 #si le déchiffrement donne bien le mot initial on considère que c'est un succès
    return ("pour un seuil de ",seuil, "on a tant de réussite",r,"et d'échec",d)

In [10]:
#fonction permettant l'automatisation des tests avec la version raffinée
def qualite_test_2(n,seuil,tour,erreur,Pk):
    r = 0
    d = 0
    for i in range (n):
        mot = Pk[randint(0,511)]
        for k in range(10):
            mot += Pk[randint(0,511)] # on génère aléatoirement un message mot
        mot2 = copy(mot)
        for j in range(erreur):
            mot[randint(0,1023)] += 1 # on génère aléatoirement le vecteur erreur pour voir notre chiffré
        chiffre = matrix(GF(2),mot)
        message = bitflipv3(H,chiffre,seuil,tour)
        if message == "erreur" :
            d = d+1
        if message == mot2 :
            r = r+1 #si le déchiffrement donne bien le mot initial on considère que c'est un succès
    return ("pour un seuil de ",seuil, "on a tant de réussite",r,"et d'échec",d)

In [43]:
#Génération de la clé privé Sk
l = [0 for i in range(1024)]
l1 = l[:512]
for i in range(15):
    l1[randint(0,511)] = 1
l2 = l[512:1024]
for i in range(15):
    l2[randint(0,511)] = 1
a,H = MatGen(l1,l2);H

512 x 1024 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)

In [44]:
#Génération de la clé public Pk
Pk = dual(a^-1*H,GF(2))

In [45]:
#génération de notre message "mot" et son chiffré "chiffre"
mot = Pk[randint(0,511)]
for k in range(10):
    mot += Pk[randint(0,511)]
for j in range(14):
    mot[randint(0,1023)] += 1
chiffre = matrix(GF(2),mot)

In [28]:
#test du temps effectué pour une correction de 14 erreurs pour un mot de longueur 1024 pour un seuil à 10
# pour la première version de notre bitflip
time(bitflipv1(H,chiffre,10))

CPU times: user 42.5 s, sys: 281 ms, total: 42.8 s
Wall time: 44.3 s


(1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 

In [22]:
#test du temps effectué pour une correction de 14 erreurs pour un mot de longueur 1024 pour un seuil à 10
time(bitflipv2(H,chiffre,10,10))

CPU times: user 968 ms, sys: 0 ns, total: 968 ms
Wall time: 973 ms


(1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 

In [23]:
#100 tests pour une matrice H de poids 30 de taille 512-1024 pour une correction de 20 erreurs avec 
#l'agorithme de bitflip optimisé
qualite_test(100,10,10,20,Pk)

('pour un seuil de ', 10, 'on a tant de réussite', 65, "et d'échec", 35)

In [24]:
#100 tests pour une matrice H de poids 30 de taille 512-1024 pour une correction de 20 erreurs avec 
#l'agorithme de bitflip raffiné
qualite_test_2(100,10,10,20,Pk)

('pour un seuil de ', 10, 'on a tant de réussite', 89, "et d'échec", 11)

In [25]:
#100 tests pour une matrice H de poids 30 de taille 512-1024 pour une correction de 15 erreurs avec 
#l'agorithme de bitflip optimisé
qualite_test(1000,10,10,15,Pk)

('pour un seuil de ', 10, 'on a tant de réussite', 999, "et d'échec", 1)

In [46]:
#100 tests pour une matrice H de poids 30 de taille 512-1024 pour une correction de 15 erreurs avec 
#l'agorithme de bitflip optimisé
qualite_test(1000,10,10,16,Pk)

('pour un seuil de ', 10, 'on a tant de réussite', 995, "et d'échec", 5)

In [31]:
#Génération de la clé privé Sk pour 1024-2048
l = [0 for i in range(2048)]
l1 = l[:1024]
for i in range(15):
    l1[randint(0,1023)] = 1
l2 = l[1024:2048]
for i in range(15):
    l2[randint(0,1023)] = 1
a,H = MatGen(l1,l2);H

1024 x 2048 dense matrix over Finite Field of size 2 (use the '.str()' method to see the entries)

In [32]:
#Génération de la clé public Pk
Pk = dual(a^-1*H,GF(2))

In [35]:
#génération de notre message "mot" et son chiffré "chiffre"
mot = Pk[randint(0,1023)]
for k in range(10):
    mot += Pk[randint(0,1023)]
for j in range(14):
    mot[randint(0,2047)] += 1
chiffre = matrix(GF(2),mot)

In [36]:
time(bitflipv1(H,chiffre,10))

CPU times: user 1min 2s, sys: 0 ns, total: 1min 2s
Wall time: 1min 2s


(0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 

In [38]:
time(bitflipv2(H,chiffre,10,10))

CPU times: user 2.11 s, sys: 0 ns, total: 2.11 s
Wall time: 2.1 s


(0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 